In [81]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import glob
from PIL import Image


In [59]:
plt.rcParams['figure.figsize'] = [12, 8]


In [82]:
# path: path to folder of images
def get_formatted_images(path, cnt):
    image_list = []
    for idx, file_name in enumerate(glob.glob(path + '/*.png')):
        if idx >= cnt:
            break

        img = np.array(Image.open(file_name))

        if len(img.shape) == 2:
            print('gray img found')
            
        h, w, d = img.shape
        min_dim = min(h, w)
        img = img[0:min_dim, 0:min_dim]

        image_list.append(img)
    return image_list

def get_labels(images, label):
    labels = [label] * len(images)
    return labels

In [86]:
# from helper import get_formatted_images, get_labels


#path = 'G:\\My Drive\\School\\6. Spring 2022\\EE379K - COMPUTER VISION\\CAPTCHA Solver\\archive\\Recaptcha Dataset\\Bicycle'
path = '/Users/andrewwhitaker/Downloads/Recaptcha Dataset/Bicycle'

num_imgs = 10
plt.figure(figsize=(20, 20))

# bicycle:
images = get_formatted_images(path, cnt=num_imgs)
print(images.shape)
labels = get_labels(images, 'bicycle')
test_images = []

for i, img in enumerate(images):
    dim = (100, 100)
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    test_images.append(resized)
    plt.subplot(5, 5, i+1)
    plt.imshow(resized, cmap='gray')
    


AttributeError: 'list' object has no attribute 'shape'

<Figure size 1440x1440 with 0 Axes>

In [61]:
batch_size = 32
img_height = 100
img_width = 100

#data_dir = 'G:\\My Drive\\School\\6. Spring 2022\\EE379K - COMPUTER VISION\\CAPTCHA Solver\\archive\\Recaptcha Dataset'
data_dir = '/Users/andrewwhitaker/Downloads/Recaptcha Dataset/'

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 238399 files belonging to 17 classes.
Using 190720 files for training.
Found 238399 files belonging to 17 classes.
Using 47679 files for validation.


In [62]:
num_classes = 5

model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [63]:
train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 100, 100, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [64]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=1
)

5960/5960 [==============================] - 218s 36ms/step - loss: 6335552512.0000 - accuracy: 0.6995 - val_loss: 19689019392.0000 - val_accuracy: 0.8198


In [69]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_3 (Rescaling)     (None, 100, 100, 3)       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 49, 49, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 47, 47, 32)        9248      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 23, 23, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 21, 21, 32)       

In [70]:
test_images_2 = np.array(test_images)

In [71]:
print(test_images_2)

[[[143 132  96 ...  83  92  97]
  [146 143 126 ...  98 113 119]
  [161 161 159 ...  99 111 115]
  ...
  [ 67  57  50 ... 134 139 137]
  [ 49  50  53 ... 112 115 111]
  [ 53  53  62 ...  92  98 101]]

 [[137 142 142 ...  62  60  66]
  [127 130 134 ...  63  69  72]
  [125 126 131 ...  65  66  65]
  ...
  [213 213 217 ... 115 144 139]
  [205 205 200 ...  89 136 156]
  [212 205 183 ...  66 103 136]]

 [[ 58  57  58 ...  63  71 110]
  [ 62  64  63 ...  51  56  81]
  [ 66  64  62 ...  40  41  48]
  ...
  [ 52  52  54 ...  64  66  63]
  [ 51  53  54 ...  69  66  65]
  [ 50  52  53 ...  72  70  71]]

 ...

 [[ 68  67  71 ...  82  56  48]
  [ 76  77  77 ...  91  69  65]
  [ 64  63  60 ...  87  66  63]
  ...
  [164 165 166 ... 149 151 152]
  [165 165 166 ... 148 151 155]
  [166 167 167 ... 147 149 153]]

 [[ 55  55  54 ...  76  82  94]
  [ 68  71  72 ...  68  83  97]
  [ 96  97 100 ...  87 107 111]
  ...
  [158 161 164 ... 137 135 131]
  [162 163 162 ... 138 138 134]
  [163 160 157 ... 138 139 1

In [68]:
predictions = model.predict(test_images_2)
score = tf.nn.softmax(predictions[0])
print(score)

ValueError: in user code:

    File "/Users/andrewwhitaker/miniforge3/envs/env_tf3/lib/python3.9/site-packages/keras/engine/training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "/Users/andrewwhitaker/miniforge3/envs/env_tf3/lib/python3.9/site-packages/keras/engine/training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/andrewwhitaker/miniforge3/envs/env_tf3/lib/python3.9/site-packages/keras/engine/training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/andrewwhitaker/miniforge3/envs/env_tf3/lib/python3.9/site-packages/keras/engine/training.py", line 1751, in predict_step
        return self(x, training=False)
    File "/Users/andrewwhitaker/miniforge3/envs/env_tf3/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Us